Uploading Phase:

In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

Calling the Neccessary Libraries:

In [2]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage import exposure
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import cv2
import glob
import os

Finding Height and Width of an Image:

In [3]:
sample_image_path = '/content/drive/MyDrive/FACE IMAGES/1.1.pgm'

# Load the image in grayscale
img = cv2.imread('/content/drive/MyDrive/FACE IMAGES/1.1.pgm', cv2.IMREAD_GRAYSCALE)

# Print its shape
print("Image shape:", img.shape)

height = img.shape[0]
print("Height:", height)

Image shape: (80, 64)
Height: 80


Training Part:

In [4]:
# ----------------------------
# CONFIGURATION
# ----------------------------
DATASET_PATH = "/content/drive/MyDrive/FACE IMAGES"
IMAGE_EXTENSION = ".pgm"
NUM_SUBJECTS = 250
TRAIN_PER_SUBJECT = 2
PATCH_SIZE = 5
stride = 5
target_size = (60, 80)

# LBP configurations: (R, P)
LBP_CONFIGS = [(1, 16), (2, 16), (1, 8), (2, 8)]
REFERENCE_CONFIG = (2, 16)

# ----------------------------
# FUNCTION: RIU2 Mapping
# ----------------------------
def get_riu2_mapping(P):
    table = np.zeros(2 ** P, dtype=np.uint8)
    for i in range(2 ** P):
        binary = [(i >> j) & 1 for j in range(P)]
        binary.append(binary[0])
        transitions = sum(binary[j] != binary[j+1] for j in range(P))
        if transitions <= 2:
            table[i] = sum(binary[:P])
        else:
            table[i] = P + 1
    return table

# ----------------------------
# FUNCTION: Extract LBP histogram
# ----------------------------
def extract_lbp_histogram(block, P, R, riu2_map):
    lbp = local_binary_pattern(block, P, R, method='ror').astype(np.uint16)
    lbp_mapped = riu2_map[lbp]
    hist, _ = np.histogram(
        lbp_mapped.ravel(),
        bins=np.arange(0, P + 3),
        density=True
    )
    return hist

# ----------------------------
# FUNCTION: Augment Image
# ----------------------------
def augment_image(img):
    h, w = img.shape
    aug_imgs = []

    # Rotations
    for angle in [-10, -5, 5, 10]:
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
        rotated = cv2.warpAffine(img, M, (w, h))
        aug_imgs.append(rotated)

    # Horizontal Flip
    flipped = cv2.flip(img, 1)
    aug_imgs.append(flipped)

    # Gaussian Blur
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    aug_imgs.append(blurred)

    return aug_imgs

# ----------------------------
# PRE-COMPUTE MAPPINGS
# ----------------------------
mapping_dict = {P: get_riu2_mapping(P) for _, P in LBP_CONFIGS + [REFERENCE_CONFIG]}

# ----------------------------
# TRAINING PHASE
# ----------------------------
train_lbp_features = []
train_labels = []

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(1, TRAIN_PER_SUBJECT + 1):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        augmented_images = augment_image(resized)
        augmented_images.append(resized)  # Include original

        for aug in augmented_images:
            denoised = cv2.fastNlMeansDenoising(aug, h=10)
            equalized = cv2.equalizeHist(denoised).astype(np.float64) / 255.0
            normalized = (equalized - np.mean(equalized)) / np.std(equalized)

            feature_vector = []
            for y in range(0, normalized.shape[0] - PATCH_SIZE + 1, stride):
                for x in range(0, normalized.shape[1] - PATCH_SIZE + 1, stride):
                    block = normalized[y:y + PATCH_SIZE, x:x + PATCH_SIZE]
                    combined_hist = []
                    for R, P in LBP_CONFIGS:
                        hist = extract_lbp_histogram(block, P, R, mapping_dict[P])
                        combined_hist.extend(hist)
                    feature_vector.extend(combined_hist)

            train_lbp_features.append(feature_vector)
            train_labels.append(subject_id)

        print(f"✅ Processed {filename} with {len(augmented_images)} images")

# Normalize training features
train_lbp_features = np.array(train_lbp_features, dtype=np.float32)
train_lbp_features = normalize(train_lbp_features, norm='l2')
train_labels = np.array(train_labels)

# ----------------------------
# MANHATTAN DISTANCE BASED RECOGNITION WITH VOTING
# ----------------------------
from collections import Counter

TEST_PER_SUBJECT = 2
TEST_START_INDEX = 3

test_labels = []
correct_matches = 0

total_tests = NUM_SUBJECTS * TEST_PER_SUBJECT

print("\n🗳️ Starting test phase using Voting + Manhattan distance...")

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_offset in range(TEST_PER_SUBJECT):
        sample_id = TEST_START_INDEX + sample_offset
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load test image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        augmented_images = augment_image(resized)
        augmented_images.append(resized)

        predictions = []

        for aug in augmented_images:
            denoised = cv2.fastNlMeansDenoising(aug, h=10)
            equalized = cv2.equalizeHist(denoised).astype(np.float64) / 255.0
            normalized = (equalized - np.mean(equalized)) / np.std(equalized)

            feature_vector = []
            for y in range(0, normalized.shape[0] - PATCH_SIZE + 1, stride):
                for x in range(0, normalized.shape[1] - PATCH_SIZE + 1, stride):
                    block = normalized[y:y + PATCH_SIZE, x:x + PATCH_SIZE]
                    combined_hist = []
                    for R, P in LBP_CONFIGS:
                        hist = extract_lbp_histogram(block, P, R, mapping_dict[P])
                        combined_hist.extend(hist)
                    feature_vector.extend(combined_hist)

            feature_vector = np.array(feature_vector, dtype=np.float32).reshape(1, -1)
            feature_vector = normalize(feature_vector, norm='l2')

            distances = np.sum(np.abs(train_lbp_features - feature_vector), axis=1)
            best_match_idx = np.argmin(distances)
            predicted_label = train_labels[best_match_idx]
            predictions.append(predicted_label)

        final_prediction = Counter(predictions).most_common(1)[0][0]
        test_labels.append(subject_id)

        print(f"🧪 Test Image {filename} | Votes: {Counter(predictions)} | Final Prediction: {final_prediction}")

        if final_prediction == subject_id:
            correct_matches += 1

accuracy = (correct_matches / total_tests) * 100
print(f"\n✅ Final Recognition Accuracy (Voting + Manhattan): {accuracy:.2f}% ({correct_matches}/{total_tests})")


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


✅ Processed 1.1.pgm with 7 images
✅ Processed 1.2.pgm with 7 images
✅ Processed 2.1.pgm with 7 images
✅ Processed 2.2.pgm with 7 images
✅ Processed 3.1.pgm with 7 images
✅ Processed 3.2.pgm with 7 images
✅ Processed 4.1.pgm with 7 images
✅ Processed 4.2.pgm with 7 images
✅ Processed 5.1.pgm with 7 images
✅ Processed 5.2.pgm with 7 images
✅ Processed 6.1.pgm with 7 images
✅ Processed 6.2.pgm with 7 images
✅ Processed 7.1.pgm with 7 images
✅ Processed 7.2.pgm with 7 images
✅ Processed 8.1.pgm with 7 images
✅ Processed 8.2.pgm with 7 images
✅ Processed 9.1.pgm with 7 images
✅ Processed 9.2.pgm with 7 images
✅ Processed 10.1.pgm with 7 images
✅ Processed 10.2.pgm with 7 images
✅ Processed 11.1.pgm with 7 images
✅ Processed 11.2.pgm with 7 images
✅ Processed 12.1.pgm with 7 images
✅ Processed 12.2.pgm with 7 images
✅ Processed 13.1.pgm with 7 images
✅ Processed 13.2.pgm with 7 images
✅ Processed 14.1.pgm with 7 images
✅ Processed 14.2.pgm with 7 images
✅ Processed 15.1.pgm with 7 images
✅ P

/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3

🧪 Test Image 1.3.pgm | Votes: Counter({np.int64(1): 6, np.int64(10): 1}) | Final Prediction: 1


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3

...

/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3

🧪 Test Image 250.3.pgm | Votes: Counter({np.int64(250): 7}) | Final Prediction: 250


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3

🧪 Test Image 250.4.pgm | Votes: Counter({np.int64(250): 7}) | Final Prediction: 250

✅ Final Recognition Accuracy (Voting + Manhattan): 96.80% (484/500)


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
